In [0]:
%pip install pyodbc


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"


In [0]:
connectionProperties = {
    'server': dbutils.secrets.get(scope="jdbc", key="server"),
    'username': dbutils.secrets.get(scope="jdbc", key="username"),
    'password': dbutils.secrets.get(scope="jdbc", key="password"),
    'host': dbutils.secrets.get(scope="jdbc", key="host"),
    'port': dbutils.secrets.get(scope="jdbc", key="port"),
    'database': 'AdventureWorks'
}

In [0]:
host= dbutils.secrets.get(scope="jdbc", key="host")
port= dbutils.secrets.get(scope="jdbc", key="port")
database= 'AdventureWorks'
username= dbutils.secrets.get(scope="jdbc", key="username")
password= dbutils.secrets.get(scope="jdbc", key="password")

In [0]:
jdbcUrl = f"jdbc:sqlserver://{host}:{port};databaseName={database}"



In [0]:
tables = [
    "PRODUCTION.Product",
    "HUMANRESOURCES.Employee",
    "PERSON.CountryRegion",
    "PRODUCTION.ProductSubcategory",
    "PRODUCTION.ProductCategory",
    "SALES.Customer",
    "SALES.SalesOrderDetail",
    "SALES.SalesOrderHeader",
    "SALES.SalesOrderHeaderSalesReason",
    "SALES.SalesPerson",
    "SALES.SalesReason",
    "SALES.SalesTerritory",
    "PERSON.Person"
]


In [0]:
target_dir = "/Volumes/ted_dev/dbt_adventureworks_raw/raw"

In [0]:
for full_table in tables:
    schema, table = full_table.split(".")
    temp_path = f"{target_dir}/_tmp_{table}"      # Temp write location
    final_path = f"{target_dir}/{table}.parquet"  # Final file path

    # Define query for JDBC read
    df = spark.read.format("jdbc") \
        .option("url", jdbcUrl) \
        .option("dbtable", full_table) \
        .option("user", username) \
        .option("password", password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .option("encrypt", "true") \
        .option("trustServerCertificate", "true") \
        .load()

    df.coalesce(1).write.mode("overwrite").parquet(temp_path)

    # List part file from temp folder
    files = dbutils.fs.ls(temp_path)
    parquet_file = [f.path for f in files if f.path.endswith(".parquet")][0]

    # Move it to final path
    dbutils.fs.cp(parquet_file, final_path)

    # Clean up temp folder
    dbutils.fs.rm(temp_path, True)